Source: 

    [1] http://www.bursamalaysia.com/market/listed-companies/list-of-companies/main-market/
    [2] https://www.morningstar.com/
    [3] https://www.klsescreener.com/
    [4] https://klse.i3investor.com/

In [1]:
import numpy as np
import pandas as pd
import re, time, os
from datetime import datetime, date
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import TimeoutException
# from time import time
from tqdm import tqdm
from glob import glob
chromedriver = 'D://Workspace/selenium_driver/bin/chromedriver'
out_dir = 'result_2023-05-21/'
if not os.path.exists(out_dir):
    os.makedirs(out_dir)

In [ ]:
stock_code = pd.read_csv(out_dir+'klse_00_stock_code.csv')
data1_1 = pd.read_csv(out_dir+'klse_01a_latest_data.csv')

# data1_1['code'] = data1_1['code'].astype(str).apply(lambda x: x.zfill(4))

### Get all listed company names and stock codes from Bursa Malaysia

In [30]:
options = webdriver.ChromeOptions()
options.add_argument("--incognito")
driver = webdriver.Chrome(chromedriver, options=options)
driver.switch_to.window(driver.current_window_handle)
driver.maximize_window()
driver.get('http://www.bursamalaysia.com/market/listed-companies/list-of-companies/main-market/')
# driver.find_element_by_class_name('cc-btn').click()
time.sleep(2)
driver.execute_script("window.scrollTo(0, document.body.scrollHeight + 200);")
time.sleep(2)
driver.find_element(By.CLASS_NAME,'custom-select').find_elements(By.TAG_NAME,'option')[-1].click()
time.sleep(2)
x = driver.find_elements(By.CLASS_NAME,'listed-company-table')[-1]\
          .find_element(By.TAG_NAME,'tbody')\
          .find_elements(By.CLASS_NAME,'company-announcement-link')

code, name = [],[]
for i in tqdm(x):
    code.append(i.get_attribute('href').split('=')[-1])
    name.append(i.get_attribute('innerText').strip())

driver.execute_script("window.scrollTo(0, 0);")
time.sleep(2)
# driver.get('https://www.bursamalaysia.com/trade/trading_resources/listing_directory/ace_market')
driver.find_element(By.LINK_TEXT,'ACE Market').click()
time.sleep(2)
driver.execute_script("window.scrollTo(0, document.body.scrollHeight + 200);")
time.sleep(2)
driver.find_element(By.CLASS_NAME,'custom-select').find_elements(By.TAG_NAME,'option')[-1].click()
time.sleep(2)
x = driver.find_elements(By.CLASS_NAME,'listed-company-table')[-1]\
          .find_element(By.TAG_NAME,'tbody')\
          .find_elements(By.CLASS_NAME,'company-announcement-link')

for i in tqdm(x):
    code.append(i.get_attribute('href').split('=')[-1])
    name.append(i.get_attribute('innerText').strip())
# driver.quit()

100%|████████████████████████████████████████████████████████████████████████████| 163/163 [00:01<00:00, 100.72it/s]


In [31]:
stock_code = pd.DataFrame({'code':code,'name':name})
idx1 = stock_code[stock_code.name == 'KLCC REAL ESTATE INVESTMENT TRUST'].index
idx2 = stock_code[stock_code.name == 'KLCC PROPERTY HOLDINGS BERHAD'].index
stock_code.loc[idx1, 'code'] = '5235SS'
stock_code.drop(index=idx2, inplace=True)
stock_code.sort_values('code', inplace=True)
stock_code.reset_index(drop=True)
print(stock_code.shape)
stock_code.to_csv(out_dir+'klse_00_stock_code.csv', index=False)
stock_code.head(1)

(951, 2)


,code,name
921,0001,SUPERCOMNET TECHNOLOGIES BERHAD


### Stage 1

In [101]:
def process_stage_1(stock_code):
    # Get stock prices in 52 weeks range from KLSE Screener, which will be used in Stage 4
    start_time = time.time()
#     options = webdriver.ChromeOptions()
#     options.add_argument("--incognito")
#     driver = webdriver.Chrome(chromedriver, options=options)
#     driver.switch_to.window(driver.current_window_handle)
#     driver.maximize_window()
    error1, result1 = [],[]
    for i in tqdm(stock_code.code):
        try:
            driver.get('https://www.klsescreener.com/v2/stocks/view/'+i)
            time.sleep(1)
            try:
                cat = driver.find_element(By.XPATH,'//*[@id="page"]/div[2]/div[1]/div[1]/div[1]/div[1]/div[1]/div[3]/span')\
                            .get_attribute('innerText')\
                            .split(' : ')[1]
            except:
                cat = 'UNKNOWN'
            price  = driver.find_element(By.ID,'price')\
                           .get_attribute('innerText')
            week52 = driver.find_element(By.XPATH,'//*[@id="page"]/div[2]/div[1]/div[2]/div[1]/div/table[1]/tbody/tr[8]/td[2]')\
                           .get_attribute('innerText')\
                           .split(' - ')
            roe    = driver.find_element(By.XPATH,'//*[@id="page"]/div[2]/div[1]/div[2]/div[1]/div/table[1]/tbody/tr[9]/td[2]')\
                           .get_attribute('innerText')\
                           .replace(',','')\
                           .replace('-','')
            roe    = float(roe) if roe else 0.0
            pe     = driver.find_element(By.XPATH,'//*[@id="page"]/div[2]/div[1]/div[2]/div[1]/div/table[1]/tbody/tr[10]/td[2]')\
                           .get_attribute('innerText')\
                           .replace(',','')
            pe     = pe if pe else 0.0
            eps    = driver.find_element(By.XPATH,'//*[@id="page"]/div[2]/div[1]/div[2]/div[1]/div/table[1]/tbody/tr[11]/td[2]')\
                           .get_attribute('innerText')\
                           .replace(',','')
            eps    = np.around(float(eps)/100, 2) if eps else 0.0
            ey     = np.around(float(eps)/float(price)*100, 2)
            dps    = driver.find_element(By.XPATH,'//*[@id="page"]/div[2]/div[1]/div[2]/div[1]/div/table[1]/tbody/tr[12]/td[2]')\
                           .get_attribute('innerText')
            dps    = np.around(float(dps)/100, 2) if dps else 0.0
            dy     = driver.find_element(By.XPATH,'//*[@id="page"]/div[2]/div[1]/div[2]/div[1]/div/table[1]/tbody/tr[13]/td[2]')\
                           .get_attribute('innerText')\
                           .replace('%','')
            bvps   = driver.find_element(By.XPATH,'//*[@id="page"]/div[2]/div[1]/div[2]/div[1]/div/table[1]/tbody/tr[14]/td[2]')\
                           .get_attribute('innerText')\
                           .replace('-','')
            bvps   = float(bvps) if bvps else 0.0

            result1.append([str(i).zfill(4), str(cat), float(price), float(week52[0]), float(week52[-1]), float(bvps),
                            float(pe), float(eps), ey, float(dps), float(dy), float(roe)])
        except Exception as e:
            error1.append([str(i).zfill(4), type(e).__name__])
#     driver.quit()
    print('Time taken: ',time.time()-start_time)
    data1 = pd.DataFrame(result1, columns=['code','category','Price','52w low','52w high','BVPS',
                                           'P/E','EPS','EY','DPS','DY','ROE'])
    er_df_1 = pd.DataFrame(error1, columns=['code','reason'])
    return data1, er_df_1

In [76]:
data1_1, er_df_1_1 = process_stage_1(stock_code)
print(data1_1.shape)
data1_1.to_csv(out_dir+'klse_01a_latest_data_part1.csv', index=False)
display(data1_1.head(1))
display(er_df_1_1.reason.value_counts())

if len(er_df_1_1) != 0:
    display(er_df_1_1)

100%|█████████████████████████████████████████████████████████████████████████████| 951/951 [53:36<00:00,  3.38s/it]


Time taken:  3216.1088140010834
(943, 12)


,code,category,Price,52w low,52w high,BVPS,P/E,EPS,EY,DPS,DY,ROE
0,0001,Health Care Equipment & Services,1.31,1.08,1.83,0.0,30.41,0.04,3.05,0.02,1.53,10.02


reason
ValueError    8
Name: count, dtype: int64

,code,reason
0,0064,ValueError
1,0111,ValueError
2,0271,ValueError
3,5238,ValueError
4,5247,ValueError
5,5622,ValueError
6,7016,ValueError
7,7053,ValueError


In [111]:
balance1_1 = stock_code[stock_code.code.isin(er_df_1_1.code)]
print('Total stock not in first batch:',balance1_1.shape)

data1_1_1, er_df_1_1_1 = process_stage_1(balance1_1)
print(data1_1_1.shape)
data1_1_1.to_csv(out_dir+'klse_01a_latest_data_part2.csv', index=False)
display(data1_1_1.head(1))
display(er_df_1_1_1.reason.value_counts())

if len(er_df_1_1_1) != 0:
    display(er_df_1_1_1)

Total stock not in first batch: (8, 2)


100%|█████████████████████████████████████████████████████████████████████████████████| 8/8 [00:26<00:00,  3.26s/it]

Time taken:  26.04440426826477
(8, 12)


,code,category,Price,52w low,52w high,BVPS,P/E,EPS,EY,DPS,DY,ROE
0,0064,Industrial Services,0.185,0.16,0.245,0.0,-3627.45,-0.0,-0.0,0.0,0.0,0.06


Series([], Name: count, dtype: int64)

In [137]:
data1_1 = pd.concat([pd.read_csv(file) for file in glob(out_dir+'klse_01a_latest_data_part*')])
data1_1.code = data1_1.code.apply(lambda x: str(x).zfill(4))
print('Total:\t',data1_1.shape)
data1_1.to_csv(out_dir+'klse_01a_latest_data.csv', index=False)
balance1_1 = stock_code[~stock_code.code.isin(data1_1.code)]
print('Balance:',balance1_1.shape)

Total:	 (951, 12)
Balance: (0, 2)


In [141]:
cat_list = ['Consumer Products & Services','Energy','Financial Services','Health Care','Industrial Products & Services',
            'Real Estate Investment Trusts','Technology','Transportation & Logistics','Utilities']
# data1_1 = data1_1[data1_1.category.isin(cat_list)]
print('Filter by Category:',data1_1.shape)

Filter by Category: (951, 12)


In [140]:
np.sort(data1_1.category.unique())

array(['Agricultural Products', 'Auto Parts', 'Automotive', 'Banking',
       'Building Materials', 'Chemicals', 'Closed End Fund',
       'Construction', 'Consumer Services', 'Digital Services',
       'Diversified Industrials', 'Electricity',
       'Energy, Infrastructure, Equipment & Services', 'Food & Beverages',
       'Gas, Water & Multi-Utilities', 'Health Care Equipment & Services',
       'Health Care Providers', 'Houshold Goods',
       'Industrial Engineering',
       'Industrial Materials, Components & Equipment',
       'Industrial Services', 'Insurance', 'Media', 'Metals',
       'Oil & Gas Producers', 'Other Energy Resources',
       'Other Financials', 'Packaging Materials', 'Personal Goods',
       'Pharmaceuticals', 'Plantation', 'Property',
       'Real Estate Investment Trusts', 'Retailers', 'Semiconductors',
       'Software', 'Special Purpose Acquisition Company',
       'Technology Equipment', 'Telecommunications Equipment',
       'Telecommunications Service Pr

### Stage 2

Earning Yield Level:
    - 5  PE <=6.67    EY >=15%         DY >=3%
    - 4  PE >6.67-10  EY 10%-<15%      DY 2.5%-<3%
    - 3  PE >10-20    EY 5%-<10%       DY 2%-<2.5%
    - 2  PE >20-30    EY 3.33%-<5%     DY 1.5%-<2%
    - 1  PE >30-40    EY 2.5%-<3.33%   DY 1%-<1.5%
    - 0  PE >4        EY <2.5%         DY <1%